In [1]:
from woocommerce import API
import os
from dotenv import load_dotenv

import re

load_dotenv()

woocommerce_url = os.getenv('URL')
woocommerce_key = os.getenv('KEY')
woocommerce_secret = os.getenv('SECRET')

wc_api = API(
    url=woocommerce_url,
    consumer_key=woocommerce_key,
    consumer_secret=woocommerce_secret,
    version='wc/v3'
)

In [2]:
params = {
    'search': "SAUCONY",
    'status': 'publish',
    'password_protected': False,
    'per_page': 95
}

In [3]:
all_products = []

page = 1
while True:
    params['page'] = page
    products = wc_api.get("products", params=params).json()
    if not products:
        break
    all_products.extend(products)
    page += 1

In [4]:
saucony_products = []

for product in all_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'Saucony' in attribute.get('options', []):
            saucony_products.append(product)


In [5]:
saucony_originals_products = []

for product in all_products:
    attributes = product.get('attributes', [])
    for attribute in attributes:
        if attribute.get('name') == 'Brend' and 'Saucony Originals' in attribute.get('options', []):
            saucony_originals_products.append(product)


In [6]:
saucony_out_of_stock_skus = []

for product in saucony_products:
    total_stock = 0
    product_id = product['id']
    variations = wc_api.get(f"products/{product_id}/variations").json()

    for variation in variations:
        stock_quantity = variation['stock_quantity']
        total_stock += stock_quantity if stock_quantity is not None else 0

    if total_stock == 0:
        saucony_out_of_stock_skus.append(product['sku'])


In [7]:
saucony_product_skus = [product['sku'] for product in saucony_products]

In [8]:
len(saucony_product_skus), len(saucony_out_of_stock_skus)

(434, 65)

In [9]:
len(set(saucony_product_skus)), len(set(saucony_out_of_stock_skus))

(433, 65)

In [10]:
from collections import Counter

counter = Counter(saucony_product_skus)

duplicate = [item for item, count in counter.items() if count > 1]
duplicate


['S20579-30']

In [11]:
active_saucony_skus = [sku for sku in saucony_product_skus if sku not in saucony_out_of_stock_skus]

In [12]:
active_saucony_products = [product for product in saucony_products if product['sku'] in active_saucony_skus]

In [13]:
active_saucony_products_wo_short_description = [product['sku'] for product in active_saucony_products if not product['short_description']]

In [14]:
active_saucony_products_wo_short_description

['SA81171-VP', 'SA81171-VCT']

In [23]:
# pattern = r'^(?!.*tenisic.{1})(?=.*mušk.{1}).*$'
# pattern = r'^(?!.*tenisic.{1})(?=.*žensk.{1}).*$'

# pattern = r'mušk.{1,}tenisic.{1,}'
# pattern = r'^(?=.*mušk.{1})(?=.*tenisic.{1}).*$'

# pattern = r'žensk.{1,}tenisic.{1,}'
pattern = r'^(?=.*žensk.{1})(?=.*tenisic.{1}).*$'

missing_skus = []

for product in active_saucony_products:
    short_description = product['short_description']
    if re.search(pattern, short_description, re.IGNORECASE):
        attributes = product.get('attributes', [])
        sizechart_attribute = next((attr for attr in attributes if attr.get('name') == 'sizechart'), None)
        if sizechart_attribute is None or 'SAUWFTW' not in sizechart_attribute.get('options'):
                    missing_skus.append(product['sku'])
                    
missing_skus

[]

In [ ]:
missing_skus = []
for product in saucony_originals_products:
    attributes = product.get('attributes', [])
    sex_found= False
    sizechart_found = False
    for attribute in attributes:
        if attribute.get('name') == 'Spol' and attribute.get('options') == ['Muškarci']:
            sex_found = True
        if attribute.get('name') == 'sizechart' and 'SAUMFTW' not in attribute.get('options'):
            sizechart_found = True
    if sex_found and sizechart_found:
        missing_skus.append(product['sku'])
missing_skus


In [ ]:
missing_skus = []
for product in saucony_originals_products:
  attributes = product.get('attributes', [])
  for attribute in attributes:
    if attribute.get('name') == 'sizechart' and attribute.get('options') is None:
      missing_skus.append(product['sku'])
missing_skus

In [ ]:
missing_skus = []

for product in saucony_originals_products:
    attributes = product.get('attributes', [])
    sex_found = False
    sizechart_found = False

    spol_attribute = next((attr for attr in attributes if attr.get('name') == 'Spol'), None)
    sizechart_attribute = next((attr for attr in attributes if attr.get('name') == 'sizechart'), None)

    if spol_attribute and sizechart_attribute:
        if spol_attribute.get('options') == ['Žene', 'Muškarci']:
            sex_found = True
        if 'SAUUFTW' not in sizechart_attribute.get('options'):
            sizechart_found = True

    if sex_found and sizechart_found:
        missing_skus.append(product['sku'])
missing_skus


In [ ]:
products = wc_api.get('products/', params = params).json()

In [ ]:
products = wc_api.get('products', params = params).json()

In [ ]:
attributes = products[0].get('attributes')
for attribute in attributes:
  if attribute['name'] == 'sizechart':
    print('yea')
attributes


In [ ]:
product_sku = 'S20768-30'
product = wc_api.get('products', params={'sku': product_sku}).json()
product

In [ ]:
existing_attributes = product["attributes"]

In [ ]:
new_attribute = {
    'id': 7,
    'name': 'sizechart',
    'position': 5,
    'visible': True,
    'variation': False,
    'options': ['SAUMFTW']
}
modified_attribute = {'id': 2,
    'options': ['Saucony Originals']
}

In [ ]:
for attribute in existing_attributes:
    if attribute['id'] == modified_attribute['id']:
        attribute.update(modified_attribute)

In [ ]:
existing_attributes.append(new_attribute)


In [ ]:
product_id = product['id']

In [ ]:
payload = {"attributes": existing_attributes}

In [ ]:
wc_api.put(f"products/{product_id}", payload).json()

In [ ]:
existing_attributes